In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/05_CNN')

In [4]:
#pip install tensorflow

In [5]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy


from sklearn.model_selection import KFold

#import pydot

In [6]:
dataset = pd.read_csv('01_Description_Train.csv', encoding='latin-1')
testdata = pd.read_csv('01_Description_Test.csv', encoding='latin-1')

train_data = dataset
test_data = testdata

#Oversampling
count_class_0, count_class_1 = train_data.Fake.value_counts()

df_class_0 = train_data[train_data['Fake'] == 0]
df_class_1 = train_data[train_data['Fake'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

df_test_over = df_test_over.reset_index(drop=True)
print('Random over-sampling:')
print(df_test_over.Fake.value_counts())

train_data = df_test_over

print(train_data.Fake.value_counts())

train_data.dropna(axis=0, how ='any', inplace=True)
train_data['Num_words_desc'] = train_data['Description'].apply(lambda x:len(str(x).split()))
#train_data['Num_words_desc'] = train_data['Description'].apply(lambda x:len(str(x).split()))
print('-------Train data--------')
print(train_data['Fake'].value_counts())
print(len(train_data))
print('-------------------------')

max_train_desc_length  = train_data['Num_words_desc'].max()
#max_train_desc_length  = train_data['Num_words_desc'].max()


test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_desc'] = test_data['Description'].apply(lambda x:len(str(x).split())) 

max_test_desc_length  = test_data['Num_words_desc'].max()

print('-------Test data--------')
print(test_data['Fake'].value_counts())
print(len(test_data))
print('-------------------------')

print('Train Max Length :'+str(max_train_desc_length))
print('Test Max Sentence Length :'+str(max_test_desc_length))


Random over-sampling:
0    5760
1    5760
Name: Fake, dtype: int64
0    5760
1    5760
Name: Fake, dtype: int64
-------Train data--------
0    5760
1    5760
Name: Fake, dtype: int64
11520
-------------------------
-------Test data--------
0    1429
1     184
Name: Fake, dtype: int64
1613
-------------------------
Train Max Length :42
Test Max Sentence Length :38


In [7]:
train_data.head()

,Description,Fake,Num_words_desc
0,enemy virus ? ongoing war COVID19 exact opposi...,0,13
1,average Chinese population spent around seven ...,0,18
2,Coronado Crime Report : Larceny Assault Deadly...,0,18
3,note size SARSCoV2 approximately nm times smal...,0,11
4,worsening economic outlook worsening economic ...,0,26


In [8]:
num_words = 20000


tokenizer = Tokenizer(num_words=num_words, oov_token="unk")
tokenizer.fit_on_texts(train_data['Description'].tolist())

print(str(tokenizer.texts_to_sequences(['5G Global bill airfare'])))

[[16, 36, 34, 7574]]


In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['Description'].tolist(),\
                                                      train_data['Fake'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['Fake'].tolist(),\
                                                      random_state=0)

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(test_data['Description'].tolist()) )

x_train = pad_sequences(x_train, padding='post', maxlen=50)
x_valid = pad_sequences(x_valid, padding='post', maxlen=50)
x_test = pad_sequences(x_test, padding='post', maxlen=50)

print(x_train[0])

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels, 2))
#print(train_labels)
valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels, 2))


test_labels = le.transform(test_data['Fake'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels, 2))
list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

Train data len:9216
Class distributionCounter({0: 4608, 1: 4608})
Valid data len:2304
Class distributionCounter({1: 1152, 0: 1152})
[ 4112  1811 12502  8011  3458  1394   628    13    38 12503  6396   359
   594 12504 12505  2680   870  4766 12506  7560   425  3896  3224 12507
  5051  7527  5624  3224 12508  5698     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_4688\2764893551.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array( tokenizer.texts_to_sequences(X_train) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_4688\2764893551.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_4688\2764893551.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuple

In [10]:
test_data.head()

,Description,Fake,Num_words_desc
0,Hepa filters remove even extremely tiny partic...,0,12
1,CORONAVIRUS known COVID19 swept north Italy we...,0,17
2,Phones Gadgets Gaming Motors Motors News Revie...,0,18
3,First Trump s wink debunked conspiracy theorie...,0,16
4,visiting COVID19 testing site please call ahea...,0,15


In [11]:
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count =0
print('======Validation dataset ====')
for value,label in valid_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count = 0
print('======Test dataset ====')
for value,label in test_ds:
    count += 1
    print(value,label)
    if count==3:
        break  


======Train dataset ====
tf.Tensor(
[ 4112  1811 12502  8011  3458  1394   628    13    38 12503  6396   359
   594 12504 12505  2680   870  4766 12506  7560   425  3896  3224 12507
  5051  7527  5624  3224 12508  5698     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0], shape=(50,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[1152  425  147    3  173  158  957  464 1784 5138   80 5139  225    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0], shape=(50,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[   22 12367  3223  8062 12368 12369   154   134  2872 12370  8017  2341
  4088   782 12371    24  5289     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0    

In [16]:
#CNN MODEL
from tensorflow.keras.optimizers.legacy import Nadam
max_features = 20000
embedding_dim = 64 #same as URLNet
sequence_length = 50

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

inputs = np.concatenate((x_train,x_valid), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)

#regularizer prevents overfitting
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    
    model = tf.keras.Sequential()
    #model.add(layers.Flatten(input_shape=(None,50,64)))
    model.add(tf.keras.layers.Embedding(max_features+1, embedding_dim, input_length=sequence_length,
                                        embeddings_regularizer = regularizers.l2(0.0005)))                                    

    model.add(tf.keras.layers.Conv1D(128, 3, activation='relu',
                                     kernel_regularizer = regularizers.l2(0.0005),
                                     bias_regularizer = regularizers.l2(0.0005)))                               


    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dropout(0.5)) #to reduce overfitting

    #final classification, 2 classes
    model.add(tf.keras.layers.Dense(2, activation='sigmoid',\
                                    kernel_regularizer=regularizers.l2(0.001),\
                                    bias_regularizer=regularizers.l2(0.001),))


    model.summary()
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='Nadam', metrics=["CategoricalAccuracy"])
    
    epochs1 = 100
    # Fit the model using the train and test datasets.

    history = model.fit(inputs[train], targets[train], epochs= epochs1 ,verbose=1)
 
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 64)            1280064   
                                                                 
 conv1d_2 (Conv1D)           (None, 48, 128)           24704     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1305026 (4.98 MB)
Trainable params: 1305026 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
________________

KeyboardInterrupt: 

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
history.history

In [ ]:
"""plt.plot(history.history['loss'], label=' training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Loss for Text Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

In [ ]:
"""plt.plot(history.history['categorical_accuracy'], label=' (training data)')
plt.plot(history.history['val_categorical_accuracy'], label='CategoricalCrossentropy (validation data)')
plt.title('CategoricalAccuracy for Text Classification')
plt.ylabel('CategoricalAccuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

In [ ]:
for layer in model.layers:
    layer._name = layer.name + str("_1")

In [ ]:
model.save('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc') 
json_string1 = tokenizer.to_json()

In [ ]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc\\Desc.json', 'w') as outfile1:
    json.dump(json_string1, outfile1)

In [ ]:
#reload model

new_model = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc')
new_model.summary()

In [ ]:
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc\\Desc.json') as json_file1:
    json_string1 = json.load(json_file1)
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string1)

In [ ]:
x_test  = np.array( tokenizer1.texts_to_sequences(test_data['Description'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=50)

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = new_model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

In [ ]:
test_data['pred_fake']= predict_results
test_data['pred_fake'] = np.where((test_data.pred_fake == '0'),0,test_data.pred_fake)
test_data['pred_fake'] = np.where((test_data.pred_fake == '1'),1,test_data.pred_fake)

In [ ]:
print(type(test_data['Fake']))
test_data['Fake'].head()


In [ ]:
print(type(test_data['pred_fake']))
test_data['pred_fake'].head()

In [ ]:
labels = [0, 1]
    
print(classification_report(test_data['Fake'].tolist(),test_data['pred_fake'].tolist(),labels=labels))

In [ ]:
"""precisionscore = precision_score(y_test, new_model.predict(x_test))
accuracyscore = accuracy_score(y_test, new_model.predict(x_test))
recallscore = recall_score(y_test, new_model.predict(x_test))
f1score = f1_score(y_test, new_model.predict(x_test))
cm = confusion_matrix(y_test, new_model.predict(x_test))

print(precisionscore, accuracyscore, recallscore, f1score, cm)
"""